In [8]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

# Test for all main scripts

In [9]:
def generate_str_config(config):
    config_list = []
    config_list.append('ue\={}'.format(config['ue']))
    if config['ue'] == 'mc-dpp' or config['ue'] == 'mc':
        config_list.append('ue.committee_size\=2')
        
    config_list.append('do_ue_estimate\=True')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    config_list.append('training.per_device_eval_batch_size\=32')

    config_list.append("model.model_name_or_path\='{}'".format(config['model_name_or_path']))
    return config_list

In [10]:
def generate_bash(config, cuda_devices, generate_func, filename):
    full_config = '# exit when any command fails\n'
    full_config += 'set -e\n'
    full_config += 'cd ../src\n'
    
    n_tests = 0
    for conf in expand_config(config):
        script = conf['script']
        task_name = conf['task']
        new_task = f'CUDA_VISIBLE_DEVICES={cuda_devices} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python {script} '
        args = ' '.join(generate_func(conf))

        reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
        use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
        ue = conf['ue']
        
        if ue=='sngp':
            if reg!='raw':
                continue
            if use_spectralnorm!='no_sn':
                continue
            if conf['model_name_or_path']!='google/electra-base-discriminator':
                continue
        if ue=='mc' or ue=='mc-dpp':
            if use_spectralnorm!='no_sn':
                continue

        new_task += f"{args}"
        if 'deberta' in conf['model_name_or_path']:
            model = 'deberta'
        elif 'electra' in conf['model_name_or_path']:
            model = 'electra'
        elif 'roberta' in conf['model_name_or_path']:
            model = 'roberta'
        elif 'distilbert' in conf['model_name_or_path']:
            model = 'distilbert'
        new_task+=f' output_dir=../workdir/run_tests/{model}_{reg}_{use_spectralnorm}/{task_name}/{ue}'

        full_config += '\n' + new_task if len(full_config) else new_task
        n_tests += 1
    print(f'n_tests: {n_tests}')
    with open (f'../../scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [11]:
train_configs = {
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.02],
    
    'task': ['cola', 'sst5'],
    'ue': ['decomposing_md', 'nuq', 'mahalanobis', 'mc', 'mc-dpp', 'sngp'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'model_name_or_path': ['microsoft/deberta-base', 'roberta-base', 
                           'google/electra-base-discriminator', 'distilbert-base-cased']
}
    
cuda_devices = '0'

generate_bash(train_configs, cuda_devices, generate_str_config, 'miscl_scripts/run_tests.sh')

n_tests: 194


In [12]:
def generate_str_config(config):
    config_list = []
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('data.subsample_perc_val\={}'.format(config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('training\=electra_base')

    config_list.append("model.model_name_or_path\='{}'".format(config['model_name_or_path']))
    return config_list

In [13]:
train_configs = {
    'script': ['run_conll2003.py'],
    'validation_subsample': [0.01],
    'subsample_perc': [0.01],
    'task': ['conll2003'],

    'ue': ['decomposing_md', 'nuq', 'mahalanobis', 'mc', 'mc-dpp', 'sngp'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False', 'True'],
    'model_name_or_path': ['microsoft/deberta-base', 'google/electra-base-discriminator', 'distilbert-base-cased']
}
    
cuda_devices = '2'

generate_bash(train_configs, cuda_devices, generate_str_config, 'ner_scripts/run_tests.sh')

n_tests: 73


In [14]:
full_test = 'nohup bash miscl_scripts/run_tests.sh > miscl_scripts/test.out &\n'
full_test += 'nohup bash ner_scripts/run_tests.sh > ner_scripts/test.out &'

with open (f'../../scripts/run_tests.sh', 'w') as rsh:
    rsh.write(full_test)